In [1]:
import pandas as pd

# Load your dataset
file_path = '/content/IndianFinancialNews[1].csv'
df = pd.read_csv(file_path)

# Display the first few rows to understand its structure
print(df.head())

# Assuming your text data is in a column named 'text_column'
texts = df['Description'].tolist()


   Unnamed: 0                    Date  \
0           0   May 26, 2020, Tuesday   
1           1   May 26, 2020, Tuesday   
2           2    May 25, 2020, Monday   
3           3    May 24, 2020, Sunday   
4           4  May 23, 2020, Saturday   

                                               Title  \
0  ATMs to become virtual bank branches, accept d...   
1  IDFC First Bank seniors to forgo 65% of bonus ...   
2  Huge scam in YES Bank for many years, says Enf...   
3  Bank of Maharashtra sanctioned Rs 2,789 cr in ...   
4  DCB Bank's profit before tax declines 37.6% to...   

                                         Description  
0  Close to 14.6 per cent (or 35,000) of the 240,...  
1  V Vaidyanathan, managing director and chief ex...  
2  Rana Kapoor's wife also charged with abetting ...  
3  The bank said it was now gearing up to extend ...  
4  Net profit for the financial year ended March ...  


In [4]:
texts

['Close to 14.6 per cent (or 35,000) of the 240,000 ATMs in India are new-age recyclers, even though they have been around for only ...',
 'V Vaidyanathan, managing director and chief executive, will take 30 per cent cut in his compensation, including fixed ...',
 "Rana Kapoor's wife also charged with abetting crime",
 'The bank said it was now gearing up to extend the stimulus package announced by Finance Minister Nirmala Sitharaman to restart ...',
 'Net profit for the financial year ended March 31, 2020 (FY20), stood at Rs 337.25 crore, up marginally from Rs 325.37 crore in ...',
 'Under the scheme, the government will offer 100 per cent guarantee on loans.',
 'HDFC Bank cuts base rate by 55 bps; SBI calls ALCO meeting to take call',
 'The six-member MPC also kept policy stance unchanged at \'accommodative\' for "as long as necessary" to revive growth',
 'The central bank has increased group exposure limits of lenders from 25% to 30%',
 'Two new series will have maturities of April 

In [1]:
import pandas as pd
import numpy as np

# Load your dataset
file_path = '/content/IndianFinancialNews[1].csv'
df = pd.read_csv(file_path)

# Replace NaN values with empty strings in 'text_column'
df['Description'] = df['Description'].fillna('')

# Display the first few rows to understand its structure
print(df.head())

# Convert 'text_column' to list
texts = df['Description'].tolist()

   Unnamed: 0                    Date  \
0           0   May 26, 2020, Tuesday   
1           1   May 26, 2020, Tuesday   
2           2    May 25, 2020, Monday   
3           3    May 24, 2020, Sunday   
4           4  May 23, 2020, Saturday   

                                               Title  \
0  ATMs to become virtual bank branches, accept d...   
1  IDFC First Bank seniors to forgo 65% of bonus ...   
2  Huge scam in YES Bank for many years, says Enf...   
3  Bank of Maharashtra sanctioned Rs 2,789 cr in ...   
4  DCB Bank's profit before tax declines 37.6% to...   

                                         Description  
0  Close to 14.6 per cent (or 35,000) of the 240,...  
1  V Vaidyanathan, managing director and chief ex...  
2  Rana Kapoor's wife also charged with abetting ...  
3  The bank said it was now gearing up to extend ...  
4  Net profit for the financial year ended March ...  


In [2]:
qna_data = []

for text in texts:
    # Ensure text is a string
    if isinstance(text, str):
        # Split text into sentences
        sentences = text.split('.')

        # Process only if there are sentences available
        if len(sentences) > 1:
            context = sentences[0].strip()
            question = "What is " + context + "?"
            answer = sentences[1].strip()
            qna_data.append((context, question, answer))


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

# Tokenizer and model initialization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Example: Convert qna_data into a dataset
qa_dataset = Dataset.from_dict({
    'context': [item[0] for item in qna_data],
    'question': [item[1] for item in qna_data],
    'answer': [item[2] for item in qna_data],
})

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples['question'], examples['context'], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_datasets = qa_dataset.map(tokenize_function, batched=True)

# Data collator
from transformers import default_data_collator

data_collator = default_data_collator

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],  # Assuming you have a validation split
    data_collator=data_collator,
)

# Train the model
trainer.train()


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/38144 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, DistilBertForQuestionAnswering, default_data_collator

# Model initialization
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Data collator
data_collator = default_data_collator

# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
)

# Train the model
trainer.train()
